In [ ]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

import os
import io
import pickle
import requests
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from tqdm import tqdm
import time

# API scope for read-only access
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def authenticate_google_drive():
    """Authenticate with Google Drive API"""
    creds = None

    # token.pickle stores authentication credentials
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If no valid credentials available, authenticate
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save credentials for next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('drive', 'v3', credentials=creds)


# In Google Colab, use default service account
from google.colab import auth
from oauth2client.client import GoogleCredentials

def authenticate_colab():
    """Authenticate in Google Colab"""
    auth.authenticate_user()
    return build('drive', 'v3')

def download_folder(service, folder_id, destination_path):
    """
    Download complete folder from Google Drive using API

    Args:
        service: Google Drive service
        folder_id: Target folder ID
        destination_path: Local save path
    """

    # Ensure destination directory exists
    os.makedirs(destination_path, exist_ok=True)

    def get_files_in_folder(folder_id):
        """Get all files in folder"""
        files = []
        page_token = None

        while True:
            try:
                # Search for files in folder
                response = service.files().list(
                    q=f"'{folder_id}' in parents and trashed=false",
                    spaces='drive',
                    fields='nextPageToken, files(id, name, mimeType, size)',
                    pageToken=page_token
                ).execute()

                files.extend(response.get('files', []))
                page_token = response.get('nextPageToken', None)

                if page_token is None:
                    break

            except Exception as e:
                print(f"[ERROR] Failed to fetch files: {e}")
                break

        return files

    def download_file(file_id, file_name, download_path):
        """Download individual file"""
        try:
            request = service.files().get_media(fileId=file_id)
            file_path = os.path.join(download_path, file_name)

            # Create subdirectories if needed
            os.makedirs(os.path.dirname(file_path), exist_ok=True)

            # Download with progress tracking
            with open(file_path, 'wb') as f:
                downloader = MediaIoBaseDownload(f, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    if status:
                        print(f'[DOWNLOAD] Downloading {file_name}: {int(status.progress() * 100)}%')

            print(f'[SUCCESS] Downloaded: {file_name}')
            return True

        except Exception as e:
            print(f'[ERROR] Failed to download {file_name}: {e}')
            return False

    def process_folder(current_folder_id, current_path):
        """Process folder and all contents recursively"""
        items = get_files_in_folder(current_folder_id)

        for item in items:
            item_name = item['name']
            item_id = item['id']
            item_type = item['mimeType']
            item_path = os.path.join(current_path, item_name)

            if item_type == 'application/vnd.google-apps.folder':
                # If folder, process recursively
                print(f'[PROCESSING] Processing folder: {item_name}')
                process_folder(item_id, item_path)
            else:
                # If file, download it
                print(f'[DOWNLOAD] Downloading file: {item_name}')
                download_file(item_id, item_name, current_path)

    # Start processing from root folder
    print(f'[START] Starting folder download to: {destination_path}')
    process_folder(folder_id, destination_path)
    print('[COMPLETE] Download completed successfully!')

# Method 1: Use Colab authentication
service = authenticate_colab()

# Method 2: If outside Colab
# service = authenticate_google_drive()

# Your folder information
FOLDER_ID = '1EksEOjw5IzGjJv1hFOh9lqvtJy7trk_W'  # Replace with your folder ID
DESTINATION = '/content/research_papers_Classification'

# Start download
download_folder(service, FOLDER_ID, DESTINATION)

In [ ]:
# [STEP 2] Copy entire downloaded folder structure to Google Drive
import os
import shutil
from google.colab import drive

# Define paths
local_directory = "/content/research_papers_Classification"
drive_path = '/content/drive'

# Check if drive is already mounted, if not then mount it
if not os.path.exists(os.path.join(drive_path, 'MyDrive')):
    print("[DRIVE] Mounting Google Drive...")
    drive.mount(drive_path)
else:
    print("[DRIVE] Google Drive is already mounted")

# Define Google Drive destination
drive_directory = "/content/drive/MyDrive/research_papers_Classification"

# Copy entire folder structure to Google Drive
print("[COPY] Copying entire folder structure to Google Drive...")
if os.path.exists(drive_directory):
    print("[REMOVE] Removing existing directory in Google Drive...")
    shutil.rmtree(drive_directory)

shutil.copytree(local_directory, drive_directory)
print(f"[SUCCESS] Entire folder structure successfully copied to Google Drive: {drive_directory}")